# NBA Score Predictor 

## Step 1: Data Collection
The URL for the website being used: https://www.basketball-reference.com/leagues/

1. Get the URL page for the match schedules and result page
2. Go through each month in the season and gather the links to the box scores (WEB SCRAPING)


In [3]:
import os 
from bs4 import BeautifulSoup # Help Parse HTML and Extract Information from it
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time 

In [6]:
!playwright install

In [4]:
SEASONS  = list(range(2016,2025)) ## Create a list of years to scrape data for

In [5]:
# Referencing these directories later

data_dir = "data" 
standings_dir = os.path.join(data_dir, "Standings") # Information scraped that will list all the box scores
scores_dir = os.path.join(data_dir, "Scores") # Directory to score all the box scores

In [ ]:
# Defining a function that takes a URL and a selector to grab the HTML from a part of the web pages

# 1. Function to get HTML

async def get_html(url, selector, sleep=3, retries=3): ## async because playwright works asynchronously because it opens a web browser which takes time 
    html = None
    for i in range(1,retries+1):
        time.sleep(sleep * 1) # To prevent scraping too fast to prevent bans from server by pausing the program for a few sleep * i seconds

        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()  # Chromium is open the open-source version of Chrome
                page = await browser.new_page() # Create a new tab in browser
                await page.goto(url) # Sending url of the page to scrape
                print(await page.title()) 
                html = await page.inner_html(selector) # Grab the HTML by passing a selector selecting only a piece of the HTML
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue   ## Go back and try again
        else:
            break
    return html


# Parsing NBA Standings Using BeautifulSoup

In [7]:
# Start Writing Code to Scrape the Web Page

# 1. Get the list of months in the season

async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter") # CSS Selector ## Scraped all the HTML anchor tags needed 

    # Gather all the anchor tags and grabbing the HREF properties from them
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]

    for url in standings_pages:
        save_path = os.path.join(standings_dir, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        # Extract the table that has the individual box scores from the standng pages
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:   # Save the path by writing the html in save_path
            f.write(html)

    
    

In [11]:
for season in SEASONS:
    await scrape_season(season) # Scrape for each season. ENSURING THAT SCARPING ONE SEASON IS DONE BEFORE MOVING ONTO THE NEXT ONE

2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com


# Parsing Box Score Links and Downloading Box Scores With BeautifulSoup

In [8]:
standings_files = os.listdir(standings_dir) # Lists all the files in the Directory

In [9]:
# Function to scrape a single standings file and save all the box scores inside the table

async def scrape_game(standings_file):
    with open(standings_file, "r") as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
   
    #links # Some contain youtube links, Team pages and etc
    #Filter out to get Box Score URLs

    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]


    # DOWNLOADING BOX SCORES WITH PLAYWRIGHT
    for url in box_scores:
        save_path = os.path.join(scores_dir, url.split("/")[-1]) # Grabbing last part of the URL
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
standings_files = [s for s in standings_files if ".html" in s]

for f in standings_files:
    filepath = os.path.join(standings_dir, f)

    await scrape_game(filepath)

In [ ]:
print("Done")